# Compare IQP results with Heuristic ones

In [1]:
import pandas as pd
from pickle import load
import plotly.graph_objects as go
import networkx as nx
import matplotlib.pyplot as plt
import math

In [2]:
RESULTS = "test_1"

## Getting results

In [3]:
iqp_results = load(open(f"./TEST_SETS/{RESULTS}_solution_iqp", "rb"))
heuristic_results = load(open(f"./TEST_SETS/{RESULTS}_solution_heuristic", "rb"))

## Getting costs

In [4]:
# Getting IQP costs
iqp_cost = dict()
iqp_data = dict()
if 'stats' in iqp_results:
    for a,r in iqp_results['stats'].items():
        iqp_data.update({a: pd.DataFrame(r)})
        iqp_cost.update({a: iqp_data[a]['COST TO REACH'].sum() + iqp_data[a]['WAIT_BEFORE'].sum()})

ValueError: DataFrame constructor not properly called!

In [ ]:
# Getting HEURISTIC costs
heu_cost = dict()
heu_data = dict()
if 'stats' in heuristic_results:
    for a,r in heuristic_results['stats'].items():
        heu_data.update({a: pd.DataFrame(r)})
        heu_cost.update({a: heu_data[a]['COST TO REACH'].sum() + heu_data[a]['WAIT_BEFORE'].sum()})

### Plot costs

In [ ]:
# Create figure
fig = go.Figure()

# Plot traces
fig.add_trace(go.Bar(name="IQP", 
                     x=[str(k) for k in iqp_cost.keys()]+['ALL'], 
                     y=list(iqp_cost.values())+[sum(iqp_cost.values())]))
fig.add_trace(go.Bar(name="HEURISTIC", 
                     x=[str(k) for k in heu_cost.keys()]+['ALL'], 
                     y=list(heu_cost.values())+[sum(heu_cost.values())]))

# Change the bar mode and axis
fig.update_layout(barmode='group',
                  xaxis_title="Agent",
                  yaxis_title="Cost",)

fig.show()

## Plotting paths

In [ ]:
def plot_model(list_of_trips, positions, lunch, limit=math.inf):
    """ Function used to plot trips graph.
        :param list_of_trips: List of edges,
        :param positions: Nodes coordinates,
        :param limit: Limit number of nodes,
    """
    # Create figure
    plt.figure(figsize=(16,9))
    G = nx.Graph()
    # Add nodes to graph
    G.add_nodes_from({pk for pk in positions.keys() if pk < limit})
    
    # Set labels dict
    node_labels = dict()
    # Set nodes positions
    for n, p in positions.items():
        if n < limit:
            G.nodes[n]['pos'] = p
            node_labels[n] = n

    # Define edges colors
    edge_colors = [plt.cm.tab20.colors[i] for i in range(len(list_of_trips))]
    
    # Define edges labels list
    edge_labels = dict()
    
    # Add edges
    for i,trip in enumerate(list_of_trips):
        for j,t in enumerate(trip):
            G.add_edge(t[0], t[1], 
                       color=edge_colors[i], 
                       alpha=0.5, 
                       weight=4)
            edge_labels.update({t: f"{j} (LUNCH)" if lunch[i][j+1] else j})
            
    # Set edges parameters 
    edges = G.edges()
    colors = [G[u][v]['color'] for u,v in edges]
    weights = [G[u][v]['weight'] for u,v in edges]
    alphas = [G[u][v]['alpha'] for u,v in edges]

    # Draw Nodes
    nx.draw_networkx_nodes(G, positions,
                           node_size=200,
                           node_color='c')
    # Draw Labels
    nx.draw_networkx_labels(G, positions, 
                            node_labels, 
                            font_size=10, 
                            font_color='k')
    # Draw Edges 
    nx.draw_networkx_edges(G, positions, 
                           alpha=0.5, 
                           width=weights, 
                           edge_color=colors)
    # Draw Edges Labels
    nx.draw_networkx_edge_labels(G, positions, 
                                 edge_labels=edge_labels)

    # Show graph
    plt.show()

## IQP paths

In [ ]:
if 'domino' in iqp_results and 'positions' in iqp_results:
    plot_model(iqp_results['domino'], 
               iqp_results['positions'], 
               [d['LUNCH_BEFORE'].astype(bool).to_list() for d in iqp_data.values()])
else:
    print("No data available!")

## Heuristic paths

In [ ]:
if 'domino' in heuristic_results and 'positions' in heuristic_results:
    plot_model(heuristic_results['domino'], heuristic_results['positions'])
else:
    print("No data available!")

## Results

### IQP results

In [ ]:
# Plot results table
for a,d in iqp_data.items():
    # Create figure
    fig = go.Figure(data=[go.Table(
    header=dict(values=list(d.columns),
                fill_color='royalblue',
                align='left',
                font=dict(color='white', size=12),),
    cells=dict(values=list(d.transpose().values),
               fill_color='lavender',
               align='left'))
    ])
    # Set title 
    fig.update_layout(title=f"Agent {a}")
    # Show figure
    fig.show()

### Heuristic results

In [ ]:
# Plot results table
for a,d in heu_data.items():
    # Create figure
    fig = go.Figure(data=[go.Table(
    header=dict(values=list(d.columns),
                fill_color='royalblue',
                align='left',
                font=dict(color='white', size=12),),
    cells=dict(values=list(d.transpose().values),
               fill_color='lavender',
               align='left'))
    ])
    # Set title 
    fig.update_layout(title=f"Agent {a}")
    # Show figure
    fig.show()